5. Expectation maximisation on a biased coin flip. Imagine that you have two biased coins, called $A$ and $B$. The probability that coin $A$ gives a head is $\theta_A$ and the probability that coin $B$ gives a head is $\theta_B$ (where $\theta_A$ and $\theta_B$ may not necessarily be 0.5 ). Imagine you have a collection of 5 datasets in which first one of the coins is selected (with probability of 0.5 ), you can call this random variable $Z$, and then the chosen coin is flipped 10 times, this random variable can be $X$. You have the obtained data in terms of $X$ (which I put below) but you do not know for a given dataset whether coin $A$ or coin $B$ was used, thus $Z$ is a hidden variable. Use an expectation maximisation procedure to iteratively converge on estimates of $\hat{\theta}_A$ and $\hat{\theta}_B$ given the data below. If it helps this data is also stored in datasets/biased_coin_flip.npy.

In [2]:
import numpy as np
from iminuit import Minuit
from iminuit.cost import UnbinnedNLL

ModuleNotFoundError: No module named 'iminuit'

In [ ]:
# read from npy file
with open('biased_coin_flip.npy', 'rb') as f:
    data = np.load(f)
    print(data)

[['T' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'T' 'H']
 ['T' 'T' 'T' 'T' 'T' 'T' 'H' 'T' 'H' 'T']
 ['H' 'T' 'T' 'H' 'H' 'H' 'H' 'H' 'H' 'H']
 ['T' 'T' 'H' 'T' 'T' 'H' 'H' 'H' 'T' 'H']
 ['H' 'T' 'H' 'H' 'T' 'T' 'H' 'H' 'H' 'H']]


In [ ]:
total_heads = [list(trials).count("H") for trials in data]

print (total_heads)

[8, 2, 8, 5, 7]


print("Total heads in each trial: ", total_heads)

In [ ]:
# set up binomial distribution

binomial = lambda k, _theta, n : np.math.factorial(n) / (np.math.factorial(k) * np.math.factorial(n - k)) * _theta ** k * (1 - _theta) ** (n - k)

In [ ]:
print (binomial(3, 0.5, 10))
print (binomial(1, 0.5, 2))

0.1171875
0.5


In [ ]:
# expectation step
# calculate the probability of heads for each coin

from types import new_class


theta_A = 0.9
theta_B = 0.5

# store the number of trials and number of flips
# in this example, N = 5, flip = 10
N, flip = int(data.shape[0]), int(data.shape[1])

q = np.zeros((N, 2))

#construct the negative log likelikehood function to minimise
def model(total_heads, theta_A, theta_B):
    theta = [theta_A, theta_B]
    total_log_likelihood = 0
    for i in range(0, N):
        for k in range (2):
            total_log_likelihood += q[i,k] * np.log((0.5 * binomial(total_heads[i], theta[k], flip))/q[i,k])
    return - total_log_likelihood

def model_tomini(total_heads, model):
    return lambda theta_A, theta_B: model(total_heads, theta_A, theta_B)

step = 0

while True:
    
    #construct the Q matrix
    theta = [theta_A, theta_B]

    for i in range(0, N):
        for k in range (2):
            # this is the posterior probability of coin k given the data
            q[i, k] = (0.5 * binomial(total_heads[i], theta[k], flip))/ (0.5 * binomial(total_heads[i], theta[1], flip) + 0.5 * binomial(total_heads[i], theta[0], flip))
    
    nll = model_tomini(total_heads, model)

    mi = Minuit(nll, theta_A=theta_A, theta_B=theta_B)

    mi.limits['theta_A'] = (0, 1)

    mi.limits['theta_B'] = (0, 1)
    
    mi.migrad()

    mi.hesse()

    new_theta_A, new_theta_B = mi.params['theta_A'].value, mi.params['theta_B'].value

    if np.abs(new_theta_A - theta_A) < 1e-7 and np.abs(new_theta_B - theta_B) < 1e-7:
        break
    
    theta_A, theta_B = new_theta_A, new_theta_B

    step += 1

print (q)
print ("total_steps: ", step)
print (theta_A, theta_B)

Step: 1
theta_A 0.7827269691438481 theta_B: 0.48215438885968814


Step: 2
theta_A 0.7588544983720655 theta_B: 0.4275402978418712


Step: 3
theta_A 0.7518475853164959 theta_B: 0.3919045037098392


Step: 4
theta_A 0.7484030603204367 theta_B: 0.3710231951381676


Step: 5
theta_A 0.7460654472862335 theta_B: 0.3596640104329538


Step: 6
theta_A 0.7445290203347834 theta_B: 0.3536125900007485


Step: 7
theta_A 0.7435965213541265 theta_B: 0.3503717250499018


Step: 8
theta_A 0.7430583329514068 theta_B: 0.3486200617703612


Step: 9
theta_A 0.742755637317406 theta_B: 0.3476671154998328


Step: 10
theta_A 0.7425875173677807 theta_B: 0.34714673200426893


Step: 11
theta_A 0.7424947122880687 theta_B: 0.3468619791291944


Step: 12
theta_A 0.7424436384924588 theta_B: 0.34670599197835916


Step: 13
theta_A 0.7424155747445383 theta_B: 0.3466204922319408


Step: 14
theta_A 0.7424001670521524 theta_B: 0.34657361321067415


Step: 15
theta_A 0.7423917115371554 theta_B: 0.34654790529073215


Step: 16
theta_